In [8]:
#-------kd-树构建------------时间复杂度O（logN）
"""
思路：1、构建kd二叉树：根据数据的数据的特征个数（Dimension）来对所有的数据排序（从小到大），
         取中位数（size-1）/2,作为根节点来划分左右子数据。
      2、编写深度优先遍历dfs：来搜索第一个最近邻点（即叶子节点），根据目标数据的
         第depth%n个特征值是否小于当前数据的第depth%n个特征值，是则向左；否则向右
      3、利用栈stack进行回溯，判断当前最小距离构建的超平面是否与父节点的超平面的另
         一侧相交，是则该父节点的另一侧，即兄弟节点可能与目标节点更近。
         
         

"""
#定义二叉树
class TreeNode:
    def __init__(self,s,d):
        self.vec=s #某个数据的特征向量
        self.Dimension=d #第d个特征向量
        self.left=None
        self.right=None
        self.father=None#父节点（搜索时需要回退）
    
    def __str__(self):
        return str(self.vec)#print 一个 Node 类时会打印其特征向量
    

#距离:L2范数
def dis(arr1,arr2):
    res=0
    for a,b in zip(arr1,arr2):
        res+=(a-b)**2
    return res**0.5


#构建kd树
def build(arr,l,father):#l:第l个特征值
    if len(arr)==0:
        return
    
    #数据量
    size=len(arr)
    #根据所有数据的第l个特征值，对所有数据arr进行从小到大排序
    arr.sort(key=(lambda x:x[l]))
    #取中位数
    mid=int((size-1)/2)

    root=TreeNode(arr[mid],l)
    root.father=father
    root.left=build(arr[:mid],(l+1)%n,root)#对左子序列进行构建kd数；(l+1)%n:每层轮流判读每个特征值
    root.right=build(arr[mid+1:],(l+1)%n,root)
    return root


#深度优先遍历
def dfs(depth,root,father,stack):
    if root == None:
        return father;

    stack.append(root)

    if target[depth%n]<root.vec[depth%n]:#若目标数据的第depth%n个特征值小于当前节点的第depth%n个特征值，则向左；否则向右
        return dfs(depth + 1, root.left, root,stack)
    else :
        return dfs(depth+1,root.right,root,stack)
    return root;


#自定义数据集
Featureset=[[1,6,2],
            [2,9,3],
            [5,1,4],
            [9,4,7],
            [4,2,6],
            [6,3,5],
            [7,2,5],
            [9,1,4]]

n=len(Featureset[0])#特性的个数

temp=Featureset

#递归构建kd树
root=build(temp,0,None)

#----搜索kd树，找到第一个当前最近点-----

target=[1,4,5]
stack=[]
nearest=dfs(0,root,root.father,stack)#找到最近邻点（叶子节点）
nearest_dis=dis(nearest.vec,target)#计算当前最近邻点距离

visited={}#用来判断兄弟节点是否已经讨论过

#用stack进行回溯
while stack[-1]!=root:
    print('STACK:',end=' ')
    for i in stack:
        print(i,end=' ')
    print()

    #先定义当前节点cur，父亲节点father，兄弟节点bro
    cur=stack[-1]
    stack.pop()
    father=cur.father
    bro=father.left
    if father.left==cur:#令bro为cur的兄弟节点，不为cur（因为是二叉树）
        bro=father.right
        
    #当前节点还没遍历过
    if visited.get(hash(cur))==None:
        #若超球体和父节点的超平面相交，则父节点的另一侧，即兄弟节点可能更接近target点
        if father.vec[father.Dimension]-target[father.Dimension]<nearest_dis:
            visited.update({hash(bro):'yes'})
            dfs(father.Dimension,bro,father,stack)

print("最近距离：",nearest_dis)
print("最近邻点：",nearest)


STACK: [5, 1, 4] [1, 6, 2] [4, 2, 6] 
STACK: [5, 1, 4] [1, 6, 2] [2, 9, 3] 
STACK: [5, 1, 4] [1, 6, 2] 
最近距离： 3.7416573867739413
最近邻点： [4, 2, 6]
